In [5]:
%load_ext autoreload
%autoreload 2


import datetime
import json
import os
import re
import shutil
import sqlite3
import sys
import time

import bs4
import numpy as np
import openpyxl
import pandas as pd
import requests
import selenium
from dateutil.parser import parse
from openpyxl import Workbook
from selenium import webdriver
from selenium.webdriver.common.by import By

sys.path.append('C:\\Users\\philc\\OneDrive\\Documents\\Programming\\Github\\general')

import southborough_stats.play_cricket_extract as ss_pce
import southborough_stats.sql_storage as ss_sql_s

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
website_inputs = {
    "email": "philcodavis@googlemail.com",
    "password": "Boroboy1",
    "start_date": "18/06/2021",
    "end_date": "30/06/2021",
}
webdriver_path = "C:/Users/philc/Documents/Python/chromedriver.exe"
downloads_path = "C:/Users/philc/Downloads"
storage_path = (
    "C:/Users/philc/Documents/Python/Southborough stats/Southborough-CC-Stats"
)
database_path = "C:/Users/philc/Documents/Python/Southborough stats/Southborough-CC-Stats/SboroCricket.db"

conn = sqlite3.connect(database_path)

In [3]:
print(website_inputs)

{'email': 'philcodavis@googlemail.com', 'password': 'Boroboy1', 'start_date': '18/06/2021', 'end_date': '30/06/2021'}


In [13]:
ss_pce.results_worksheet_download(website_inputs, webdriver_path, downloads_path)

In [14]:
results_path = ss_pce.move_fixture_download(
    website_inputs, downloads_path, storage_path
)

In [11]:
 ss_pce.fixture_id_extract(results_path)

NameError: name 'results_path' is not defined

In [13]:
(
    match_figures_dict1,
    match_reference_dict1,
    batting_dict1,
    bowling_dict1,
    extras_dict1,
    fall_of_wicket_dict1,
    error_msg1,
) = ss_pce.gameid_full_extract('4819228')

In [26]:
batting_dict1

{'batter_1': {'unique_batter_reference': 'SUN2021062700SOUKEMBAT1',
  'unique_match_reference': 'SUN2021062700',
  'batting_team': 'Southborough CC Sunday XI',
  'bowling_team': 'Kemsing CC Sunday 1st XI',
  'batter_number': '1',
  'batter_name': 'Phil Davis',
  'captain': 'Y',
  'wicketkeeper': 'Y',
  'secondary_dismissal': 'c',
  'secondary_dismisser': 'Jake Wood',
  'bowled': 'b',
  'bowler': 'George Scully',
  'runs': 64,
  'balls': 0.0,
  'fours': 8.0,
  'sixes': 3.0,
  'strike_rate': ''},
 'batter_2': {'unique_batter_reference': 'SUN2021062700SOUKEMBAT2',
  'unique_match_reference': 'SUN2021062700',
  'batting_team': 'Southborough CC Sunday XI',
  'bowling_team': 'Kemsing CC Sunday 1st XI',
  'batter_number': '2',
  'batter_name': 'Hassan Hadi',
  'captain': '',
  'wicketkeeper': '',
  'secondary_dismissal': 'c',
  'secondary_dismisser': 'Jake Wood',
  'bowled': 'b',
  'bowler': "Connor O'Shaughnessy",
  'runs': 19,
  'balls': 0.0,
  'fours': 3.0,
  'sixes': 0.0,
  'strike_rate':

In [9]:
# gameid = "4819228" # friendly 2021
gameid = "4994947"  # league game 2022
(
    gameid_r4,
    gameid_soup4,
    match_figures_dict,
    match_reference_dict,
    bowling_dict,
    error_msg,
) = gameid_full_extract(gameid)
innings_id = ss_pce.find_innings_id(gameid_r4.text)
innings_id

soup4 = gameid_soup4

In [10]:
innings_id

['6334557', '6339767']

In [53]:
fall_of_wicket_extract(soup4, gameid, innings_id, match_figures_dict1, match_reference_dict1, error_msg1)

({'innings_1': {'unique_fall_of_wicket_reference': '1ST2022071600SOPL',
   'unique_match_reference': '1ST2022071600',
   'batting_team': 'Southborough CC 1st XI',
   'bowling_team': 'Plaxtol CC 1st XI',
   'team_runs_1': 4,
   'batter_1': 'Phil Davis',
   'not_out_batter_1': 'Martyn Budd',
   'not_out_batter_score_1': 4,
   'team_runs_2': 34,
   'batter_2': 'Imran Jameel',
   'not_out_batter_2': 'Martyn Budd',
   'not_out_batter_score_2': 11,
   'team_runs_3': 69,
   'batter_3': 'Martyn Budd',
   'not_out_batter_3': 'Tom Williams',
   'not_out_batter_score_3': 17,
   'team_runs_4': 84,
   'batter_4': 'Tom Williams',
   'not_out_batter_4': 'Graeme Sharpe',
   'not_out_batter_score_4': 5,
   'team_runs_5': 119,
   'batter_5': 'Jack Roberts',
   'not_out_batter_5': 'Graeme Sharpe',
   'not_out_batter_score_5': 28,
   'team_runs_6': 121,
   'batter_6': 'Graeme Sharpe',
   'not_out_batter_6': 'James Rumbles',
   'not_out_batter_score_6': 2,
   'team_runs_7': 127,
   'batter_7': 'James Rumbl

In [52]:
def fall_of_wicket_extract(soup4, gameid, innings_id, match_figures_dict, match_reference_dict, error_msg):
    
    fall_of_wicket_dict = {}
    
    all_elems = soup4.select('body > div.container100sm.container.container-scorecard > div > div.col-sm-12.col-scorecard.mb50')
    
    for k in range(len(innings_id)):
        
        if (
            str(all_elems[0].find_all("a")[k].contents[0]).replace("\n", " ").strip()
            in match_figures_dict["home_team"]
        ):
            batting_team = match_figures_dict["home_team"]
            bowling_team = match_figures_dict["away_team"]
        else:
            batting_team = match_figures_dict["away_team"]
            bowling_team = match_figures_dict["home_team"]
            
        fall_of_wicket_dict[f"innings_{str(k+1)}"] = {
            "unique_fall_of_wicket_reference": "",
            "unique_match_reference": "",
            "batting_team": "",
            "bowling_team": "",
            "team_runs_1": "",
            "batter_1": "",
            "not_out_batter_1": "",
            "not_out_batter_score_1": "",
            "team_runs_2": "",
            "batter_2": "",
            "not_out_batter_2": "",
            "not_out_batter_score_2": "",
            "team_runs_3": "",
            "batter_3": "",
            "not_out_batter_3": "",
            "not_out_batter_score_3": "",
            "team_runs_4": "",
            "batter_4": "",
            "not_out_batter_4": "",
            "not_out_batter_score_4": "",
            "team_runs_5": "",
            "batter_5": "",
            "not_out_batter_5": "",
            "not_out_batter_score_5": "",
            "team_runs_6": "",
            "batter_6": "",
            "not_out_batter_6": "",
            "not_out_batter_score_6": "",
            "team_runs_7": "",
            "batter_7": "",
            "not_out_batter_7": "",
            "not_out_batter_score_7": "",
            "team_runs_8": "",
            "batter_8": "",
            "not_out_batter_8": "",
            "not_out_batter_score_8": "",
            "team_runs_9": "",
            "batter_9": "",
            "not_out_batter_9": "",
            "not_out_batter_score_9": "",
            "team_runs_10": "",
            "batter_10": "",
            "not_out_batter_10": "",
            "not_out_batter_score_10": "",
        }
        
        fall_of_wicket_dict[f"innings_{str(k+1)}"]["unique_match_reference"] = match_reference_dict["unique_match_reference"]
        fall_of_wicket_dict[f"innings_{str(k+1)}"]["batting_team"] = batting_team
        fall_of_wicket_dict[f"innings_{str(k+1)}"]["bowling_team"] = bowling_team
        fall_of_wicket_dict[f"innings_{str(k+1)}"]["unique_fall_of_wicket_reference"] = (fall_of_wicket_dict[f"innings_{str(k+1)}"]["unique_match_reference"] + fall_of_wicket_dict[f"innings_{str(k+1)}"]["batting_team"][:2].upper() + fall_of_wicket_dict[f"innings_{str(k+1)}"]["bowling_team"][:2].upper())
        
        fow_elems = soup4.select('#innings' + innings_id[k] + ' > div:nth-child(2) > div > p.font-3')
        fow_teamruns = [""]*10
        fow_batter = [""]*10
        fow_no_batter = [""]*10
        fow_no_batterscore = [""]*10
        
        if len(fow_elems) > 0:
            for i in range(len(fow_elems[0].contents)):

                if re.search('>-\d{1,2}<',str(fow_elems[0].contents[i])) != None:
                    break

                if re.search('\d{1,3}-\d{1,2}',str(fow_elems[0].contents[i])) != None:
                    wicketno = int(re.search('-\d{1,2}',str(fow_elems[0].contents[i]))[0][1:])
                    fow_teamruns[wicketno - 1] = int(re.search('\d{1,3}-',str(fow_elems[0].contents[i]))[0][:-1])

                if str(fow_elems[0].contents[i]) == " (":
                    fow_batter[wicketno - 1] = str(fow_elems[0].contents[i -1].contents[0])
                    fow_no_batter[wicketno - 1] = str(fow_elems[0].contents[i + 1].contents[0])
                    fow_no_batterscore[wicketno - 1] = int(re.search('-\d{1,3}\*', str(fow_elems[0].contents[i + 2]))[0][1:-1])
                if "Unsure (Unsure-0*)" in fow_elems[0].contents[i]:
                    fow_batter[wicketno - 1] = 'Unsure'
                    fow_no_batter[wicketno - 1] = 'Unsure'
                    fow_no_batterscore[wicketno - 1] = int(re.search('-\d{1,3}\*', fow_elems[0].contents[i])[0][1:-1])
            
        for i in range(10):
            fall_of_wicket_dict[f"innings_{str(k+1)}"][f"team_runs_{str(i+1)}"] = fow_teamruns[i]
            fall_of_wicket_dict[f"innings_{str(k+1)}"][f"batter_{str(i+1)}"] = fow_batter[i]
            fall_of_wicket_dict[f"innings_{str(k+1)}"][f"not_out_batter_{str(i+1)}"] = fow_no_batter[i]
            fall_of_wicket_dict[f"innings_{str(k+1)}"][f"not_out_batter_score_{str(i+1)}"] = fow_no_batterscore[i]
    
    return fall_of_wicket_dict, error_msg
    

In [45]:
for i in range(10):    
    print(f'"team_runs_{str(i+1)}": "",')
    print(f'"batter_{str(i+1)}": "",')
    print(f'"not_out_batter_{str(i+1)}": "",')
    print(f'"not_out_batter_score_{str(i+1)}": "",')

"team_runs_1": "",
"batter_1": "",
"not_out_batter_1": "",
"not_out_batter_score_1": "",
"team_runs_2": "",
"batter_2": "",
"not_out_batter_2": "",
"not_out_batter_score_2": "",
"team_runs_3": "",
"batter_3": "",
"not_out_batter_3": "",
"not_out_batter_score_3": "",
"team_runs_4": "",
"batter_4": "",
"not_out_batter_4": "",
"not_out_batter_score_4": "",
"team_runs_5": "",
"batter_5": "",
"not_out_batter_5": "",
"not_out_batter_score_5": "",
"team_runs_6": "",
"batter_6": "",
"not_out_batter_6": "",
"not_out_batter_score_6": "",
"team_runs_7": "",
"batter_7": "",
"not_out_batter_7": "",
"not_out_batter_score_7": "",
"team_runs_8": "",
"batter_8": "",
"not_out_batter_8": "",
"not_out_batter_score_8": "",
"team_runs_9": "",
"batter_9": "",
"not_out_batter_9": "",
"not_out_batter_score_9": "",
"team_runs_10": "",
"batter_10": "",
"not_out_batter_10": "",
"not_out_batter_score_10": "",


In [40]:
elems = soup4.select('#innings' + innings_id[k] + ' > div:nth-child(2) > div > p.font-3')
len(elems[0].contents)

56

In [ ]:
for k in range(len(innings_id)):

    FoW_figures = [""] * 44
    elems = soup4.select('body > div.container100sm.container.container-scorecard > div > div.col-sm-12.col-scorecard.mb50')
    if str(elems[0].find_all('a')[k].contents[0]).replace('\n', ' ').strip() in Teams_data[0]:
        batting_team = Teams_data[0]
        bowling_team = Teams_data[1]
    else:
        batting_team = Teams_data[1]
        bowling_team = Teams_data[0]
    FoW_figures[1:4] = [Reference_data[1], batting_team, bowling_team]
    FoW_figures[0] = FoW_figures[1] + FoW_figures[2][:2].upper() + FoW_figures[3][:2].upper()

    elems = soup4.select('#innings' + innings_id[k] + ' > div:nth-child(2) > div > p.font-3')
    FoW_teamruns = [""]*10
    FoW_batter = [""]*10
    FoW_NObatter = [""]*10
    FoW_NObatterscore = [""]*10
    if len(elems) > 0:
        for i in range(len(elems[0].contents)):

            #print(elems[0].contents[i])
            if re.search('>-\d{1,2}<',str(elems[0].contents[i])) != None:
                break

            if re.search('\d{1,3}-\d{1,2}',str(elems[0].contents[i])) != None:
                wicketno = int(re.search('-\d{1,2}',str(elems[0].contents[i]))[0][1:])
                FoW_teamruns[wicketno - 1] = int(re.search('\d{1,3}-',str(elems[0].contents[i]))[0][:-1])

            if str(elems[0].contents[i]) == " (":
                FoW_batter[wicketno - 1] = str(elems[0].contents[i -1].contents[0])
                FoW_NObatter[wicketno - 1] = str(elems[0].contents[i + 1].contents[0])
                FoW_NObatterscore[wicketno - 1] = int(re.search('-\d{1,3}\*', str(elems[0].contents[i + 2]))[0][1:-1])
            if "Unsure (Unsure-0*)" in elems[0].contents[i]:
                FoW_batter[wicketno - 1] = 'Unsure'
                FoW_NObatter[wicketno - 1] = 'Unsure'
                FoW_NObatterscore[wicketno - 1] = int(re.search('-\d{1,3}\*', elems[0].contents[i])[0][1:-1])

    for i in range(10):
        FoW_figures[4*i + 4 : 4*i + 8] = [FoW_teamruns[i], FoW_batter[i], FoW_NObatter[i], FoW_NObatterscore[i]]

    if insert_db:
        sql_insert_fow(conn, FoW_figures) 
    else:
        print(FoW_figures)